In [ ]:

from threading import Lock, Semaphore
import collections
from concurrent import futures


    

In [ ]:
# dbx solutions already added to ds+a

class MultiWebCrawler:
    def __init__(self):
        pass
    def get_domain(self, url):
        new_domain = url.split('http://')[1].split('/')[0]
    def crawl(self, startUrl, htmlParser):
        domain = self.get_domain(startUrl)
        visited = set([startUrl])
        with futures.ThreadPoolExecutor(max_workers=16) as executor:
            queue = collections.deque([executor.submit(htmlParser.getUrls, startUrl)])
            while queue:
                url_list = queue.popleft().result()
                for url in url_list:
                    new_domain = self.get_domain(url)
                    if url not in visited and new_domain == domain:
                        visited.add(url)
                        queue.append(executor.submit(htmlParser.getUrls, startUrl))
        return list(visited)
    
class FizzBuzz:
    def __init__(self,n):
        self.n = n
        self.fizzLock = Lock()
        self.buzzLock = Lock()
        self.fizzbuzzLock = Lock()
        self.fizzLock.acquire()    
        self.buzzLock.acquire()    
        self.fizzbuzzLock.acquire()    
        self.nLock = Lock()
    def fizz(self, printFizz):
        while True:
            self.fizz.acquire()
            printFizz()
            self.nLock.release()
    def buzz(self, printBuzz):
        while True:
            self.buzz.acquire()
            printBuzz()
            self.nLock.release()
    def fizzbuzz(self, printFizzBuzz):
        while True:
            self.fizzbuzz.acquire()
            printFizzBuzz()
            self.nLock.release()
    def number(self, printFizz): # main orchestrator thread 
        for i in range(1, self.n+1):
            self.nLock.acquire()
            if i%15 == 0:
                self.fizzbuzzLock.release()
            elif i%3 == 0:
                self.fizzLock.release()
            elif i%5 == 0:
                self.buzzLock.release()
            else:
                printNumber(i)
                self.nLock.release()
        self.nLock.acquire()
        self.fizz.release()
        self.buzz.release()
        self.fizzbuzz.release()
        return        

# preferred 
class BoundedBlockingQueue(object):
    def __init__(self, capacity):
        self.enqueueLock = Lock()
        self.dequeueLock = Lock()
        self.capacity = capacity
        self.queue = deque()
        self.dequeueLock.acquire() # cant pop if empty
    def enqueue(self, element):
        self.enqueueLock.acquire()
        self.queue.append(element)
        if len(self.queue) < self.capacity: # queue not full
            self.enqueueLock.release()
        if self.dequeueLock.locked(): # queue is no longer empty
            self.dequeueLock.release()
    def dequeue(self):
        self.dequeueLock.acquire()
        value = self.queue.popleft()
        if len(self.queue) > 0:
            self.dequeueLock.release()
        if value and self.enqueue.locked(): # queue is no longer full
            self.enqueue.release()
        return value        
    def size(self):
        return len(self.queue)

# not preferred but may want to learn
class BoundedBlockingQueue(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.size = 0
        self.queue = collections.deque()
        self.condition = threading.Condition()
    def enqueue(self, element): # add element to queue if not full
        self.condition.acquire()
        while self.size >= self.capacity:
            self.condition.wait()
        self.queue += element
        self.size += 1
        self.condition.notifyAll()
        self.condition.release()
    def dequeue(self): # return last element in queue
        self.condition.acquire()
        while self.size == 0:
            self.condition.wait()
        result = self.queue.popleft()
        self.size -= 1
        self.condition.notifyAll()
        self.condition.release()
        return result
    def size(self):
        return self.size    
    
    
class PrintInOrder:
    def __init__(self):
        self.lock1 = Lock()
        self.lock2 = Lock()
        self.lock1.acquire()
        self.lock2.acquire()
    def first(self, printFirst):
        printFirst()
        self.lock1.release()
    def second(self, printSecond):
        self.lock1.acquire()
        printSecond()
        self.lock2.release()
    def third(self, printThird):
        self.lock2.acquire()
        printThird()        
    
class H2O:
    def __init__(self):
        self.hLock = Lock()
        self.oLock = Lock()
        self.oLock.acquire()
        self.hcount = 0
    def hydrogen(self, releaseHydrogen):
        self.hLock.acquire()
        self.count += 1
        releaseHydrogen()
        if self.count == 2:
            self.count = 0
            self.oLock.release()
        else:
            self.hLock.release()
    def oxygen(self, releaseOxygen):
        self.oLock.acquire()
        releaseOxygen()
        self.oLock.release()
        
class ZeroEvenOdd:
    def __init__(self, n):
        self.n = n
        self.lockZ = Lock()
        self.lockE = Lock()
        self.lockO = Lock()
        self.lockO.acquire()
        self.lockE.acquire()        
    def zero(self, printNumber):
        for i in range(0, self.n+1):
            self.lock0.acquire()
            printNumber(0)
            if i%2 == 1:
                self.lockOdd.release()
            else:
                self.lockEven.release()            
    def even(self, printNumber):
        for j in range(2, self.n+1, 2):
            self.lockEven.acquire()
            printNumber(j)
            self.lock0.release()
    def odd(self, printNumber):
        for k in range(1, self.n+1, 2):
            self.lockOdd.acquire()
            printNumber(k)
            self.lock0.release()        
class FooBar:
    def __init__(self,n):
        self.n = n
        self.fLock = Lock()
        self.bLock = Lock()
        self.bLock.acquire()
    def foo(self, printFoo):
        for i in range(0, self.n):
            self.fLock.acquire()            
            printFoo()
            self.bLock.release()
    def bar(self, printBar):
        for i in range(0, self.n):
            self.bLock.acquire()
            printBar()
            self.fLock.release()
        
class TrafficLight:
    def __init__(self):
        self.lock = Lock()
        self.roadIdGreen = 1
    def carrArrived(self, carId, roadId, direction, turnGreen, crossCar):
        self.lock.acquire()
        if roadID != self.roadIdGreen:
            self.roadIdGreeen = roadId
            turnGreen()
        crossCar()
        self.lock.release()
        
# deadlock can only occur on 0 or 1 fork, or if all five have picked up a fork
# one solution is to ensure that at most 4 philosophers can pick up forks
# 2nd solution is make philospher 0 prefer right fork 1st, everyone else prefer left fork first
# so at most 4 forks are picked up at a time 
class DiningPhilosophers:
    def __init__(self):
        self.forks = [threading.Lock() for id in range(0, 5)]    
    def wantsToEat(self, left_id, pickLeftFork, pickRightFork, eat, putLeftFork, putRightFork):
        left_id, right_id = left_id, (left_id+1)%5
        if left_id == 0: # 0 prefers right then left 
            left_id, right_id = right_id, left_id
        with self.forks[left_id]:
            with self.forks[right_id]:
                pickLeftFork()
                pickRightFork()
                eat()
                putLeftFork()
                putRightFork()

    
# dbx solutions already added to ds+a
    

In [ ]:

# do undertand 
class Foo:
    def __init__(self):
        # two and three start blocked 
        self.two = Semaphore(0) # blocks if negative 
        self.three = Semaphore(0)

    def first(self, printFirst):
        printFirst()
        self.two.release()

    def second(self, printSecond):
        with self.two:
            printSecond()
            self.three.release()

    def third(self, printThird):
        with self.three:
            printThird()

class Foo:
    def __init__(self):
        self.gates = (Semaphore(0),Semaphore(0))
        
    def first(self, printFirst):
        printFirst()
        self.gates[0].release()
        
    def second(self, printSecond):
        with self.gates[0]:
            printSecond()
            self.gates[1].release()
            
    def third(self, printThird):
        with self.gates[1]:
            printThird()        
            

            
# using Event - set() -> True, clear() -> False, wait() waits until flag is True            
class Foo:
    def __init__(self):
        self.done = (Event(),Event())
        
    def first(self, printFirst):
        printFirst()
        self.done[0].set()
        
    def second(self, printSecond):
        self.done[0].wait()
        printSecond()
        self.done[1].set()
            
    def third(self, printThird):
        self.done[1].wait()
        printThird()
            
            
        

In [ ]:

import threading
import time

mutex = threading.Lock()  # is equal to threading.Semaphore(1)

def fun1():
    while True:
        mutex.acquire()
        print(1)
        mutex.release()
        time.sleep(.5)

def fun2():
    while True:
        mutex.acquire()
        print(2)
        mutex.release()
        time.sleep(.5)

t1 = threading.Thread(target=fun1).start()
t2 = threading.Thread(target=fun2).start()

import threading
import time

mutex = threading.Lock()  # is equal to threading.Semaphore(1)

def fun1():
    while True:
        with mutex:
            print(1)
        time.sleep(.5)

def fun2():
    while True:
        with mutex:
            print(2)
        time.sleep(.5)

t1 = threading.Thread(target=fun1).start()
t2 = threading.Thread(target=fun2).start()


In [ ]:

import asyncio

def hello_world(loop):
    print('Hello World')
    loop.stop()

loop = asyncio.get_event_loop()

loop.call_soon(hello_world, loop)

loop.run_forever()
loop.close()


In [ ]:
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
values = [2,3,4,5]
def square(n):
    return n*n
def main():
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = executor.map(square, values)
    for result in results:
        print(result)
if __name__ == '__main__':
    main()

In [ ]:

import concurrent.futures
import urllib.request

URLS = ['http://www.foxnews.com/',
   'http://www.cnn.com/',
   'http://europe.wsj.com/',
   'http://www.bbc.co.uk/',
   'http://some-made-up-domain.com/']

def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
    print(future_to_url)
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page is %d bytes' % (url, len(data)))

In [ ]:
import threading
import queue
import random
import time
def myqueue(queue):
    while not queue.empty():
        item = queue.get()
        if item is None:
            break
        print("{} removed {} from the queue".format(threading.current_thread(), item))
        queue.task_done()
        time.sleep(2)

q = queue.Queue()
for i in range(5):
    q.put(i)
print(q)
threads = []
for i in range(4):
    thread = threading.Thread(target=myqueue, args=(q,))
    thread.start()
    threads.append(thread)
for thread in threads:
    thread.join()

In [ ]:
import collections
import heapq
import threading
import queue

q = queue.Queue()
q.put(val)
q.get()



#x = 0
def increment_global():
    global x
    x += 1
    
def taskofThread(lock):
    for i in range(0, 1000000):
        lock.acquire()
        increment_global()    
        lock.release()

def main():
    global x
    x = 0
    lock = threading.Lock()
    t1 = threading.Thread(target= taskofThread, args=(lock,))
    t2 = threading.Thread(target= taskofThread, args=(lock,))

    t1.start()
    t2.start()

    t1.join()
    t2.join()    

if __name__ == "__main__":
    for i in range(10):
        main()
        print("x = {0} after Iteration {1}".format(x,i))

In [ ]:
import collections
import heapq
import threading
class Solution:

    
    def trap(self, height):
        # fill to height of the left
        # drain down to height of fill to the right 
        volume = []
        left = 0
        for ht in height:
            left = max(left, ht)
            volume.append(left)
        volume.reverse()
        height.reverse()
        right = 0
        total_water = 0
        for index, ht in enumerate(height):
            right = max(right, ht)
            total_water += min(volume[index], right) - ht
        return total_water    
    
    
height = [0,1,0,2,1,0,1,3,2,1,2,1]                
                
answer = Solution()
print(answer.trap(height))

